In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def find_keypoints_and_descriptors(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(desc1, desc2):
    matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = matcher.match(desc1, desc2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

def draw_matches(img1, kp1, img2, kp2, matches):
    return cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

def stereo_calibrate(kp1, kp2, matches, img1_shape):
    # Convert keypoints to an array of points
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros_like(points1)

    for i, match in enumerate(matches):
        points1[i, :] = kp1[match.queryIdx].pt
        points2[i, :] = kp2[match.trainIdx].pt

    # Camera calibration
    focal_length = img1_shape[1]
    center = (img1_shape[1]/2, img1_shape[0]/2)
    camera_matrix = np.array([[focal_length, 0, center[0]],
                            [0, focal_length, center[1]],
                            [0, 0, 1]], dtype=np.float32)

    # Assuming no lens distortion
    dist_coeffs = np.zeros(4)

    # Stereo calibration
    ret, mtx1, dist1, mtx2, dist2, R, T, E, F = cv2.stereoCalibrate(
        [points1, points2], camera_matrix, dist_coeffs, camera_matrix, dist_coeffs,
        img1_shape[::-1], None, None, criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5),
        flags=cv2.CALIB_FIX_INTRINSIC)
    
    return mtx1, dist1, mtx2, dist2, R, T

# Load images
img1 = cv2.imread('path_to_left_image.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('path_to_right_image.jpg', cv2.IMREAD_GRAYSCALE)

# Find keypoints and descriptors
kp1, desc1 = find_keypoints_and_descriptors(img1)
kp2, desc2 = find_keypoints_and_descriptors(img2)

# Match features
matches = match_features(desc1, desc2)

# Optionally draw matches
matched_img = draw_matches(img1, kp1, img2, kp2, matches)

# Stereo calibration
mtx1, dist1, mtx2, dist2, R, T = stereo_calibrate(kp1, kp2, matches, img1.shape)

print("Intrinsic Matrix for Camera 1:", mtx1)
print("Intrinsic Matrix for Camera 2:", mtx2)
print("Rotation Matrix:", R)
print("Translation Vector:", T)

# Show matched image
cv2.imshow('Matches', matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
